<a href="https://colab.research.google.com/github/Jersae/Google-TimeSeries-workshop/blob/main/Chee_Wee_CodeAlong_TSSD_CC_Fraud_Imbalanced_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget -q https://www.dropbox.com/s/fljaie1ajeze8vy/creditcard.csv

In [2]:
ls

creditcard.csv              fraud_model_at_epoch_21.h5
creditcard.csv.1            fraud_model_at_epoch_22.h5
creditcard.csv.2            fraud_model_at_epoch_23.h5
creditcard.csv.3            fraud_model_at_epoch_24.h5
creditcard.csv.4            fraud_model_at_epoch_25.h5
creditcard.csv.5            fraud_model_at_epoch_26.h5
creditcard.csv.6            fraud_model_at_epoch_27.h5
creditcard.csv.7            fraud_model_at_epoch_28.h5
fraud_model_at_epoch_10.h5  fraud_model_at_epoch_29.h5
fraud_model_at_epoch_11.h5  fraud_model_at_epoch_2.h5
fraud_model_at_epoch_12.h5  fraud_model_at_epoch_30.h5
fraud_model_at_epoch_13.h5  fraud_model_at_epoch_3.h5
fraud_model_at_epoch_14.h5  fraud_model_at_epoch_4.h5
fraud_model_at_epoch_15.h5  fraud_model_at_epoch_5.h5
fraud_model_at_epoch_16.h5  fraud_model_at_epoch_6.h5
fraud_model_at_epoch_17.h5  fraud_model_at_epoch_7.h5
fraud_model_at_epoch_18.h5  fraud_model_at_epoch_8.h5
fraud_model_at_epoch_19.h5  fraud_model_at_epoch_9.h5
fraud_model_at_epo

In [3]:
#imports
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Dropout, Flatten

In [4]:
df = pd.read_csv('./creditcard.csv')

df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [5]:
labels = df.pop('Class')
print(labels.value_counts())

0    284315
1       492
Name: Class, dtype: int64


In [6]:
features = df.values
#cols = ['Time', 'V7', 'V8']

#df[cols].values.tolist()

features.shape

(284807, 30)

In [7]:
labels = labels.to_numpy().reshape(len(labels), 1)

labels.shape

(284807, 1)

In [8]:
# train test split
split = int(len(features) * 0.8)

x_train, y_train = features[:split], labels[:split]
x_test, y_test = features[split:], labels[split:]

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((227845, 30), (56962, 30), (227845, 1), (56962, 1))

## Normalization

In [9]:
# old fashioned way
"""
x_train_old = x_train
mean = np.mean(x_train_old, axis = 0)
x_train_old -+ mean
std = np.std(x_train_old, axis=0)
x_train_old /= std

mean, std, x_train_old
"""

#use tf.keras.layers.experimental.preprocessing.Normalization
#use adapt

'\nx_train_old = x_train\nmean = np.mean(x_train_old, axis = 0)\nx_train_old -+ mean\nstd = np.std(x_train_old, axis=0)\nx_train_old /= std\n\nmean, std, x_train_old\n'

In [10]:
#new implementation (Seems like it is consisten with old)
layer_tryout = tf.keras.layers.experimental.preprocessing.Normalization()
layer_tryout.adapt(x_train)
layer_tryout.mean, layer_tryout.variance, layer_tryout

(<tf.Variable 'mean:0' shape=(30,) dtype=float32, numpy=
 array([ 7.9042797e+04, -6.7180790e-02, -1.3512367e-02,  1.8250893e-01,
         4.3790787e-02, -6.3728966e-02,  3.0536966e-02, -2.6842874e-02,
         3.9854795e-03,  2.2204972e-03, -1.7066291e-03,  7.6276384e-02,
        -4.4996548e-02,  1.6711891e-02,  3.2869250e-02,  4.9113471e-02,
        -5.5079609e-03,  1.5156597e-02, -2.2870937e-02, -7.2854543e-03,
         9.9472860e-03, -6.6195866e-03, -2.2912972e-02, -9.9146282e-03,
         1.1059723e-03,  3.8057819e-02,  2.8380959e-03,  2.2927098e-04,
         1.9619404e-03,  9.0824921e+01], dtype=float32)>,
 <tf.Variable 'variance:0' shape=(30,) dtype=float32, numpy=
 array([1.5607127e+09, 3.7771056e+00, 2.7487762e+00, 2.2213550e+00,
        1.9917129e+00, 1.8854131e+00, 1.7461371e+00, 1.5085671e+00,
        1.4631439e+00, 1.2620698e+00, 1.1966792e+00, 1.0681466e+00,
        1.1148453e+00, 1.0395586e+00, 9.3261760e-01, 8.7219071e-01,
        7.8433996e-01, 7.6417398e-01, 7.1028268e

## Deal with weight imbalance

In [11]:
counts = np.bincount(y_train[:,0])

counts[1], 100*float(counts[1]/len(y_train))

(417, 0.18301915776075842)

In [12]:
company_weighting = 1.0

weight_for_0 = 1.0/counts[0] * company_weighting
weight_for_1 = 1.0/counts[1] * company_weighting

print("fraud weighting ", weight_for_1, 'Normal', weight_for_0)

fraud weighting  0.002398081534772182 Normal 4.3969959723516896e-06


## The model

In [13]:
layer = tf.keras.layers.experimental.preprocessing.Normalization(name="normalisation")
layer.adapt(x_train)

Inp = Input(shape=[30,1], name='Input')
x = Flatten(name='Flatten_01')(Inp)
x = layer(x)
x = Dense(256, activation='relu', name='dense_01')(x)
x = Dense(256, activation='relu', name='dense_02')(x)
x = Dropout(0.3, name='Dropout_01')(x)
x = Dense(256, activation='relu', name='dense_03')(x)
x = Dropout(0.3, name='Dropout_02')(x)
outputs = Dense(1, activation='sigmoid', name='output')(x)

model = tf.keras.Model(Inp, outputs, name='CC_Fraud_model')

In [14]:
model.summary()

Model: "CC_Fraud_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 30, 1)]           0         
_________________________________________________________________
Flatten_01 (Flatten)         (None, 30)                0         
_________________________________________________________________
normalisation (Normalization (None, 30)                61        
_________________________________________________________________
dense_01 (Dense)             (None, 256)               7936      
_________________________________________________________________
dense_02 (Dense)             (None, 256)               65792     
_________________________________________________________________
Dropout_01 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_03 (Dense)             (None, 256)            

In [15]:
metrics = [tf.keras.metrics.FalseNegatives(name='fn'), tf.keras.metrics.FalsePositives(name='fp'), tf.keras.metrics.TrueNegatives(name='tn'), tf.keras.metrics.TruePositives(name='tp'), tf.keras.metrics.Precision(name='Precision'), tf.keras.metrics.Recall(name='recall')]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss='binary_crossentropy', metrics=metrics)

callbacks = [tf.keras.callbacks.ModelCheckpoint("fraud_model_at_epoch_{epoch}.h5")]

class_weights = {0:weight_for_0, 1:weight_for_1}

In [16]:
model.fit(x_train, y_train, batch_size=2048, epochs=30, callbacks=callbacks, validation_data=(x_test , y_test), verbose=2, class_weight=class_weights)

Epoch 1/30
112/112 - 3s - loss: 2.3001e-06 - fn: 44.0000 - fp: 26629.0000 - tn: 200799.0000 - tp: 373.0000 - Precision: 0.0138 - recall: 0.8945 - val_loss: 0.1940 - val_fn: 6.0000 - val_fp: 2420.0000 - val_tn: 54467.0000 - val_tp: 69.0000 - val_Precision: 0.0277 - val_recall: 0.9200
Epoch 2/30
112/112 - 1s - loss: 1.5335e-06 - fn: 38.0000 - fp: 8289.0000 - tn: 219139.0000 - tp: 379.0000 - Precision: 0.0437 - recall: 0.9089 - val_loss: 0.0953 - val_fn: 8.0000 - val_fp: 1067.0000 - val_tn: 55820.0000 - val_tp: 67.0000 - val_Precision: 0.0591 - val_recall: 0.8933
Epoch 3/30
112/112 - 1s - loss: 1.1802e-06 - fn: 34.0000 - fp: 5808.0000 - tn: 221620.0000 - tp: 383.0000 - Precision: 0.0619 - recall: 0.9185 - val_loss: 0.0401 - val_fn: 9.0000 - val_fp: 663.0000 - val_tn: 56224.0000 - val_tp: 66.0000 - val_Precision: 0.0905 - val_recall: 0.8800
Epoch 4/30
112/112 - 1s - loss: 1.0511e-06 - fn: 20.0000 - fp: 7219.0000 - tn: 220209.0000 - tp: 397.0000 - Precision: 0.0521 - recall: 0.9520 - val_lo

In [17]:
loss, fn,fp,tn,tp, precision, recall = model.evaluate(x_test,y_test)

1781/1781 [==============================] - 7s 4ms/step - loss: 0.0657 - fn: 9.0000 - fp: 884.0000 - tn: 56003.0000 - tp: 66.0000 - Precision: 0.0695 - recall: 0.8800


In [18]:
print('Total correct fraud cases found:', int(tp),' out of ',int(tp+fn), ' cases of fraud' )

Total correct fraud cases found: 66  out of  75  cases of fraud


In [19]:
print('Total of incorrect fraud cases found:', int(fp),' out of ',int(fp+tn), ' good transactions' )

Total of incorrect fraud cases found: 884  out of  56887  good transactions


72/75 3060 30 epoch Company 2.0  
68/75 796 30 epoch Company 1.5  
68/75 934 30 epoch Company 1.7  
65/75 175 30 epoch Company 1.0  


In [20]:
ls

creditcard.csv              fraud_model_at_epoch_21.h5
creditcard.csv.1            fraud_model_at_epoch_22.h5
creditcard.csv.2            fraud_model_at_epoch_23.h5
creditcard.csv.3            fraud_model_at_epoch_24.h5
creditcard.csv.4            fraud_model_at_epoch_25.h5
creditcard.csv.5            fraud_model_at_epoch_26.h5
creditcard.csv.6            fraud_model_at_epoch_27.h5
creditcard.csv.7            fraud_model_at_epoch_28.h5
fraud_model_at_epoch_10.h5  fraud_model_at_epoch_29.h5
fraud_model_at_epoch_11.h5  fraud_model_at_epoch_2.h5
fraud_model_at_epoch_12.h5  fraud_model_at_epoch_30.h5
fraud_model_at_epoch_13.h5  fraud_model_at_epoch_3.h5
fraud_model_at_epoch_14.h5  fraud_model_at_epoch_4.h5
fraud_model_at_epoch_15.h5  fraud_model_at_epoch_5.h5
fraud_model_at_epoch_16.h5  fraud_model_at_epoch_6.h5
fraud_model_at_epoch_17.h5  fraud_model_at_epoch_7.h5
fraud_model_at_epoch_18.h5  fraud_model_at_epoch_8.h5
fraud_model_at_epoch_19.h5  fraud_model_at_epoch_9.h5
fraud_model_at_epo